In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')

Selecting previously unselected package libta-lib0.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 8.8MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437795 sha256=faadea53c41eace5673640ac02daa1a6dbb7117258cb8e89bc532da54661bd84
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib
Mounted at /

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
# gpu environment
import tensorflow as tf
from tensorflow.keras import *
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    # tf.config.experimental.set_virtual_device_configuration(gpu0,
    # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

# useful functions
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
    onehot = [0] * 2
    onehot[ int(classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

# stocknum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
start = '2012/05/02'
end = '2013/05/01'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

# for stockno in stockNum:
for stockno in ["1301"]:
    # load 進 五大價格 一個量
    df = pdr.DataReader(str(stockno)+'.TW', 'yahoo', start=start, end=end)
    df.index = df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    # print(df.columns)
    stock_time = str(df.index[0]).split(" ")[0]
    #load in 融資融券
    url = "https://api.finmindtrade.com/api/v3/data"
    parameter = {
        "dataset": "TaiwanStockMarginPurchaseShortSale",
        "stock_id": str(stockno),
        "date": stock_time,
    }
    data = requests.get(url, params=parameter)
    data = data.json()
    data = pd.DataFrame(data['data'])
    #if 沒資料 則跳過這張股票
    if data.empty == True:
        print(str(stockno)+" can't find data !")
        continue
    ratio = []
    b_total = []
    s_total = []
    for i in range(len(data)):
        b_total.append(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i])
        s_total.append(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])
        ratio.append(float(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])/(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i]+1))
    data['b_total'] = b_total
    data['s_total'] = s_total
    data['b_s_ratio'] = ratio
    data = data.set_index('date')
    #合併這兩個表格
    new_df = df.join(data)
    new_df = new_df.drop(['stock_id'], axis=1)
    new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
    #處理三大法人投資情況 視情況加入
    url = "https://api.finmindtrade.com/api/v3/data"
    parameter = {
        "dataset": "InstitutionalInvestorsBuySell",
        "stock_id": str(stockno),
        "date": stock_time,
    }
    data = requests.get(url, params=parameter)
    data = data.json()
    data = pd.DataFrame(data['data'])
    if data.empty == True:
        print(str(stockno)+" can't find data !")
        continue
    df = pd.DataFrame()
    z = 0 
    i = 0
    while i < len(data)  :
        a = data['date'][i]
        count = 0
        sell  = 0
        buy = 0 
        if i+6 < len(data)-1:
            for j  in range(i, i+6):
                if data['date'][j]== a :
                    count += 1
                    sell += data['sell'][j]
                    buy += data['buy'][j]
        else:
            for j in range(i, len(data)):
                if data['date'][j] == a :
                    count += 1
                    sell += data['sell'][j]
                    buy += data['buy'][j]
        df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
        i+=count
    # combine
    new_df = new_df.join(df.set_index('date'))
    new_df = new_df.dropna(axis=0,how='any')
    print(new_df.head(6))
    print(new_df.tail(6))
    new_df.reset_index(drop=True, inplace=True)
    new_df = new_df.drop(['Note'], axis=1)
    for columnName in new_df.columns:
        if str(columnName) == 'y_label' : continue
        new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    pred_days = 5
    ans = []
    for i in range(len(new_df)-pred_days):
        Pi = new_df['Close'][i+pred_days]
        Pj = new_df['Close'][i]
        if Pi > Pj : y = 1
        else: y = 0
        ans.append(y)
    ans += [0]*pred_days
    new_df['y_label']= ans
    #print(new_df.tail(6))
    #store file
    new_df = new_df[5:]
    #print(new_df.tail(6))
"""
    from google.colab import files
    #new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/testyears/'+str(stockno)+'.csv')
    print(str(stockno)+" complete....")

    def normalization(df):
    #{'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}
      dic = {}
      for col in df.columns:
          if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
          else:
              mean = statistics.mean(df[str(col)])
              stdev = statistics.stdev(df[str(col)])
              if stdev == 0: stdev = 1
          dic[str(col)+'_mean'] = mean
          dic[str(col)+'_stdev'] = stdev
      return dic
    record = pd.DataFrame()
    days_before = 30
    for i in range(days_before, len(new_df)):
      record = record.append(normalization(new_df[i-days_before:i]), ignore_index=True )
    #record.to_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/testyears/'+str(stockno)+'mean&stdev.csv')
    print(str(stockno)+" mean & stdev complete....")
"""

In [ ]:
def convertData(df, record):
    x = []
    df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
    df = df.drop(['Date'], axis=1)
    df = df.drop(['MarginPurchaseLimit'], axis=1)
    df = df.drop(['ShortSaleLimit'], axis=1)
    df = df.dropna(axis=0, how='any')
    df.reset_index(drop=True, inplace=True)
    days_before = 30
    for i in range(days_before, len(df)):
        tmp = df[i-days_before:i]
        for col in df.columns:
          if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
          mean = record[str(col)+'_mean'][i-30]
          stdev = record[str(col)+'_stdev'][i-30]
          # mean = statistics.mean(df[str(col)])
          # stdev = statistics.stdev(df[str(col)])
          def test(s): 
            return (float(s)-mean)/stdev
          tmp[str(col)].map(test)
        x.append( np.array(np.array(tmp).tolist()[0: days_before]).reshape(days_before,len(tmp.columns)))
    y = list(df['y_label'][days_before:])
    # convert to one-hot encoding
    y_res = [] 
    for i in range(len(y)):
      y_res.append(to_onelist(str(y[i])))
    return x, y_res

def count_label_dist(arr):
    count_1 = 0 
    count_0 = 0 
    for i in range(len(arr)):
        if list(arr[i])== list([0,1]): count_1 += 1
        elif list(arr[i])== list([1,0]): count_0 += 1
    return count_0, count_1

In [ ]:
def lstm4_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(25, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
pd.options.mode.chained_assignment = None 

train_years = ["2012to2013", "2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018"]
validation_years = ["2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018", "2018to2019"]
for train_year, validation_year in zip(train_years, validation_years):
  print(train_year, validation_year)
  train_x = []
  train_y = []
  val_x   = []
  val_y   = []
  for stockno in stockNum: 
      if int(stockno) == 2227 or int(stockno) == 2633 or int(stockno) == 3711: continue
      train_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + train_year + '/' + str(stockno)+'.csv')
      train_record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + train_year + '/' + str(stockno)+'mean&stdev.csv')
      tmp_train_x, tmp_train_y = convertData(train_df, train_record)
      valid_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + validation_year + '/' + str(stockno)+'.csv')
      valid_record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + validation_year + '/' + str(stockno)+'mean&stdev.csv')
      tmp_val_x, tmp_val_y = convertData(valid_df, valid_record)
      print(str(stockno)+" loaded....")
      print(len(tmp_train_x), len(tmp_train_y), len(tmp_val_x), len(tmp_val_y))
      train_x += tmp_train_x
      train_y += tmp_train_y
      val_x += tmp_val_x
      val_y += tmp_val_y
  train_x = np.array(train_x).reshape(len(train_x), 30, 23)
  val_x   = np.array(val_x).reshape(len(val_x), 30, 23)
  train_y = np.array(train_y).reshape(len(train_y),2)
  val_y   = np.array(val_y).reshape(len(val_y),2)
  print(count_label_dist(train_y))
  print(count_label_dist(val_y))
  print(train_year + "start-training")
  %load_ext tensorboard
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model = lstm4_model((30, 23), 2)
  # callback = EarlyStopping(monitor="val_loss", patience=50, verbose=1, mode="auto")
  # history = model.fit(train_x, train_y , epochs=300, batch_size= 256, validation_data=(val_x, val_y), callbacks=[callback, tensorboard_callback])
  history = model.fit(train_x, train_y , epochs=300, batch_size= 256, validation_data=(val_x, val_y), callbacks=[tensorboard_callback])
  model.save('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_' + train_year + '.h5')
  print(train_year + "training done")

In [ ]:
%tensorboard --logdir logs

In [ ]:
# get testing accuracy of ensemble model
# load in testing data and testing label
test_year = "2019to2020"
for stockno in stockNum: 
  if int(stockno) == 2227: continue
  test_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + test_year + '/' + str(stockno)+'.csv')
  test_record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + test_year + '/' + str(stockno)+'mean&stdev.csv')
  test_x, test_y = convertData(test_df, test_record)
  test_x = np.array(test_x).reshape(len(test_x), 30, 23)
  test_y = np.array(test_y).reshape(len(test_y), 2)
# load in models
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2012to2013.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2013to2014.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2014to2015.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2015to2016.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2016to2017.h5')
model6 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2017to2018.h5')
# start testing and calculate accuracy
truth_y = []
for i in range(len(test_y)):
  truth_y.append(int(classes[np.argmax(test_y[i])]))
prediction_y = [0]*len(test_y)
for i in range(len(test_y)):
  prediction_y[i] += int(classes[np.argmax(model1.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model2.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model3.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model4.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model5.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model6.predict(test_x[i].reshape(1,30,23)))])
  if prediction_y[i] >= 3 : prediction_y[i] = 1
  else: prediction_y[i] = 0
print("Accuracy: ")
print( 1 - (np.sum(np.logical_xor(np.array(prediction_y), np.array(truth_y))))/len(test_y) )

Accuracy: 
0.8845070422535212


In [ ]:
# stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
#             "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
#             "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
#             "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
#             "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# starts = ["2013/05/09", "2014/05/09", "2015/05/11", "2016/05/10", "2017/05/09", "2018/05/09"]
# ends = ["2014/05/02", "2015/04/30", "2016/04/29", "2017/05/02", "2018/05/02", "2019/05/02"]
# training_years = ["2012to2013", "2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018"]
# validation_years = ["2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018", "2018to2019"]
# SequentialValROR = {}
# for start, end, training_year, validation_year in zip(starts, ends, training_years, validation_years):
#   print(start, end, training_year, validation_year)
#   s = start.split('/')
#   e = end.split('/')
#   start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
#   end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))
#   model = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_' + training_year + '.h5')
#   asset_at_the_start = 51 * 10**6
#   total_asset_present_value = 0
#   ROR_Validation = {}
#   for stock_num in stockNum:
#   # for stock_num in ["1101"]:
#     Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
#     Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
#     validation_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/' + validation_year + '/' + str(stock_num)+'.csv')
#     print("data loaded")
#     new_df = validation_df
#     print(new_df.tail())

#     new_df = new_df.reset_index()
#     new_df.rename(columns={'index':'Date'}, inplace=True)
#     time_list = list(new_df['Date'])
#     # new_df = new_df.drop(['Date'], axis=1)
#     # new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
#     # new_df = new_df.drop(['ShortSaleLimit'], axis=1)
#     # new_df = new_df.drop(['Unnamed: 0'], axis=1)
#     Price_df = Price_df.reset_index()
#     print(Price_df.tail())

#     y_preds_list=[]
#     y_truth_list=[]
#     left_money = asset_at_the_start*(1/51)
#     own_asset = left_money 
#     time_slide = 30 
#     own_stock = 0 
#     money_record = [left_money]*time_slide
#     origin_hold_stock = 0 

#     for i in range(time_slide, len(new_df)): 
#       tmp = new_df[i-time_slide:i]
#       for col in new_df.columns:
#         if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
#         mean = statistics.mean(tmp[str(col)])
#         stdev = statistics.stdev(tmp[str(col)])
#         if stdev == 0: stdev = 1
#         def test(s): 
#           return (float(s)-mean)/stdev
#         tmp[str(col)].map(test)
#       x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
#       predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
#       y_truth_list.append(new_df['y_label'][i])
#       y_preds_list.append(predict)
#       price = int(Price_df['Close'][i]*100)/100
#       if predict == 1:
#         if own_stock == 0:
#           own_stock += int( left_money / ( price * (1 + 0.001425)) )
#           left_money = left_money - own_stock*price*(1 + 0.001425)
#       elif predict == 0 :
#         if own_stock > 0:
#           left_money += own_stock*price*(1 - 0.001425 - 0.003)
#           own_stock = 0
#       money_record.append(left_money+own_stock*price)
#     print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
#     total_asset_present_value += int(money_record[-1])
#     ROR_Validation[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
#     print(stock_num)
#     print(ROR_Validation[stock_num])
#     SequentialValROR[validation_year] = ROR_Validation
#   print(validation_year)
#   print("總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")
# print(SequentialValROR)
# """

Calculate Validation Rate of Return 計算六年驗證資料集的投資報酬率

In [ ]:
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", 
            "5880", "6505", "9904", "9910"]
starts = ["2013/05/09", "2014/05/09", "2015/05/11", 
          "2016/05/10", "2017/05/09", "2018/05/09"]
ends = ["2014/05/02", "2015/04/30", "2016/04/29",
        "2017/05/02", "2018/05/02", "2019/05/02"]
training_years = ["2012to2013", "2013to2014", "2014to2015",
                  "2015to2016", "2016to2017", "2017to2018"]
validation_years = ["2013to2014", "2014to2015", "2015to2016",
                    "2016to2017", "2017to2018", "2018to2019"]
SequentialValROR = {}
for start, end, training_year, validation_year in zip(starts, ends, training_years, validation_years):
  print(start, end, ",training model year =", training_year, ", validation data year =", validation_year)
  s = start.split('/')
  e = end.split('/')
  start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
  end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))
  model = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_' + training_year + '.h5')
  asset_at_the_start = 49 * 10**6
  total_asset_present_value = 0
  ROR_Validation = {}
  for stock_num in stockNum:
    # Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
    # # Return_Test_Stock_df = Price_df.copy(deep=True)
    # Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    # stock_time = str(Price_df.index[0]).split(" ")[0]
    # url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
    # parameter = {
    #   "dataset": "TaiwanStockMarginPurchaseShortSale",
    #   "stock_id": str(stock_num),
    #   "date": stock_time,
    # }
    # data = requests.get(url, params=parameter)
    # data = data.json()
    # data = pd.DataFrame(data['data'])
    # data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
    # data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
    # data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
    # data = data.set_index('date')
    # # 合併這兩個表格
    # new_df = Price_df.join(data)
    # new_df = new_df.drop(['stock_id'], axis=1)
    # new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
    # url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
    # parameter = {
    #   "dataset": "InstitutionalInvestorsBuySell",
    #   "stock_id": str(stock_num),
    #   "date": stock_time,
    # }
    # data = requests.get(url, params=parameter)
    # data = data.json()
    # data = pd.DataFrame(data['data'])
    # df = pd.DataFrame()
    # z = 0 
    # i = 0
    # while i < len(data)  :
    #   a = data['date'][i]
    #   count = 0
    #   sell = 0
    #   buy = 0 
    #   if i+6 < len(data)-1:
    #       for j  in range(i, i+6):
    #           if data['date'][j]== a :
    #             count += 1
    #             sell += data['sell'][j]
    #             buy += data['buy'][j]
    #   else:
    #     for j in range(i, len(data)):
    #       if data['date'][j] == a :
    #         count += 1
    #         sell += data['sell'][j]
    #         buy += data['buy'][j]
    #   df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    #   i+=count
    # new_df = new_df.join(df.set_index('date'))
    # new_df = new_df.fillna(axis=0,method='ffill')
    # new_df = new_df.drop(['Note'], axis=1)
    # new_df.reset_index(drop=True, inplace=True)
    # for columnName in new_df.columns:
    #   if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    #   new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    #   days = 10 # need to be 5
    #   ans = [0]*days
    #   for i in range(days,len(new_df)):
    #     Pi = new_df['Close'][i]
    #     Pj = new_df['Close'][i-days]
    #     if Pi > Pj : y = 1
    #     else: y = 0
    #     ans.append(y)
    # new_df['y_label']= ans
    # new_df = new_df.reset_index()
    # new_df.rename(columns={'index':'Date'}, inplace=True)
    # time_list = list(new_df['Date'])
    # new_df = new_df.drop(['Date'], axis=1)
    # new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
    # new_df = new_df.drop(['ShortSaleLimit'], axis=1)
    # Price_df = Price_df.reset_index()
    # new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/Validation_' + validation_year + '/' + str(stock_num) + 'new_df.csv')
    # Price_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/Validation_' + validation_year + '/' + str(stock_num) + 'price_df.csv')
    new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/Validation_' + validation_year + '/' + str(stock_num) + 'new_df.csv')
    Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Ensemble/Validation_' + validation_year + '/' + str(stock_num) + 'price_df.csv')
    if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
    if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
    # Return_Test_Stock_df.reset_index()
    # Return_Test_Stock_df = Return_Test_Stock_df[30:]
    # print(len(Price_df) == len(new_df))
    #############################################################################
    time_slide = 30 
    y_preds_list=[0]*(time_slide-1)
    # y_truth_list=[]
    left_money = asset_at_the_start*(1/51)
    own_asset = left_money 
    
    own_stock = 0 
    money_record = [left_money]*time_slide
    origin_hold_stock = 0 
    for i in range(time_slide, len(new_df)): 
      tmp = new_df[i-time_slide:i]
      for col in new_df.columns:
        if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
        mean = statistics.mean(tmp[str(col)])
        stdev = statistics.stdev(tmp[str(col)])
        if stdev == 0: stdev = 1
        def test(s): 
          return (float(s)-mean)/stdev
        tmp[str(col)].map(test)
      x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
      predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
      # y_truth_list.append(new_df['y_label'][i])
      y_preds_list.append(predict)
    for i in range(time_slide, len(new_df)): 
      price = int(Price_df['Close'][i]*100)/100
      if y_preds_list[i-4] == 1:
        if own_stock == 0:
          own_stock += int( left_money / ( price * (1 + 0.001425)) )
          left_money = left_money - own_stock*price*(1 + 0.001425)
      elif y_preds_list[i-4] == 0 :
        if own_stock > 0:
          left_money += own_stock*price*(1 - 0.001425 - 0.003)
          own_stock = 0
      money_record.append(left_money+own_stock*price)
    #print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
    total_asset_present_value += int(money_record[-1])
    ROR_Validation[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
    print("\"" + str(stock_num) + "\"" + ":", end = "")
    print(ROR_Validation[stock_num])
    print(", ", end="")
    SequentialValROR[validation_year] = ROR_Validation
  #print(validation_year)
  # print("總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")
  print(SequentialValROR[validation_year])
print(SequentialValROR)

2013/05/09 2014/05/02 ,training model year = 2012to2013 , validation data year = 2013to2014
"1101":0.08632602040816321
, "1102":-0.07716957142857146
, "1216":-0.08528481632653065
, "1301":0.1965942653061224
, "1326":-0.008374734693877596
, "1402":0.0002692448979591385
, "2002":0.0022561632653060773
, "2105":-0.18283428571428575
, "2207":-0.037161632653061265
, "2301":-0.06114620408163269
, "2303":-0.14038251020408166
, "2308":0.1489321632653061
, "2317":0.1337008571428571
, "2327":0.3755865714285714
, "2330":-0.14150555102040818
, "2352":0.36576126530612235
, "2357":0.00096867346938771
, "2382":-0.1467897755102041
, "2395":0.16198399999999996
, "2408":-0.14112357142857146
, "2412":-0.10028089795918371
, "2454":0.1270635714285714
, "2474":0.4167550204081632
, "2609":-0.029538693877551064
, "2610":-0.030901122448979638
, "2801":-0.0782822040816327
, "2823":-0.1383206530612245
, "2880":-0.030135081632653105
, "2881":-0.12289471428571432
, "2882":-0.14516402040816329
, "2883":-0.0759924081

Convert Validation ROR to Testing Asset Allocations 將驗證資料集的投資報酬率轉成測試資料得投資組合分佈

In [ ]:
這些是預測35天漲跌30天買賣的

# ##2013/05/09 2014/05/02 ,training model year = 2012to2013 , validation data year = 2013to2014
# dict_1314 = {"1101": 0.085478, "1102": -0.04748, "1216": -0.008218, "1301": 0.004213, "1326": 0.025005000000000003, "1402": -0.136284, "2002": -0.097525, "2105": -0.013768,
# "2207": 0.18027200000000002, "2301": -0.234631, "2303": 0.085366, "2308": 0.064972, "2317": 0.080144, "2327": 0.31522,
# "2330": 0.079063, "2352": 0.11302300000000001, "2357": 0.23427199999999998, "2382": 0.163342, "2395": 0.049181, "2408": -0.10364000000000001, "2412": -0.121255,
# "2454": 0.060306, "2474": 0.301863, "2609": -0.031976, "2610": -0.033377, "2801": -0.036392, "2823": -0.016068, "2880": -0.050206999999999995, "2881": -0.161019, 
# "2882": -0.047247000000000004, "2883": -0.0030120000000000004, "2884": -0.141817, "2885": -0.073564, "2886": -0.099833, "2887": -0.126277, "2888": -0.13633699999999999,
# "2890": -0.137637, "2891": 0.00243, "2892": -0.031432, "2912": 0.18438400000000002, "3008": 0.11979799999999999, "3045": -0.10726200000000001, 
# "4904": 0.01145, "4938": -0.233334, "5871": -0.18451599999999999, "5880": -0.089554, "6505": -0.07566200000000001, "9904": 0.060035, "9910": 0.453434}

# #2014/05/09 2015/04/30 ,training model year = 2013to2014 , validation data year = 2014to2015
# dict_1415 = {"1101": -0.090756, "1102": -0.08095599999999999, "1216": 0.019461, "1301": -0.07437, "1326": -0.060762000000000004, "1402": -0.078458, "2002": -0.11341200000000001,
# "2105": -0.089865, "2207": 0.078741, "2301": -0.179477, "2303": -0.166447, "2308": -0.110653, "2317": -0.036009, "2327": 0.192796, "2330": 0.005849, "2352": -0.059463999999999996,
# "2357": -0.049455, "2382": -0.180788, "2395": 0.088559, "2408": -0.221508, "2412": -0.003836, "2454": -0.295076, "2474": 0.139645, "2609": 0.047446, "2610": 0.187228, 
# "2801": -0.124141, "2823": -0.13999499999999998, "2880": 0.037312, "2881": 0.27954799999999996, "2882": 0.08104499999999999, "2883": 0.169998, "2884": -0.10841300000000001, 
# "2885": -0.184097, "2886": -0.028828, "2887": -0.053741000000000004, "2888": -0.018932, "2890": 0.09290599999999999, "2891": 0.215363, "2892": 0.009198, "2912": -0.204211,
# "3008": 0.41324399999999994, "3045": -0.113139, "4904": -0.043006, "4938": 0.38023, "5871": 0.151407, "5880": -0.10687300000000001, "6505": -0.092741, "9904": 0.221375, "9910": 1.255873}

# #2015/05/11 2016/04/29 ,training model year = 2014to2015 , validation data year = 2015to2016
# dict_1516 = {"1101": -0.143506, "1102": -0.160304, "1216": 0.033502000000000004, "1301": -0.106271, "1326": 0.09102, "1402": -0.073978, "2002": -0.045113, "2105": -0.089484, 
# "2207": -0.295347, "2301": -0.088668, "2303": -0.163242, "2308": -0.393083, "2317": -0.207748, "2327": -0.20996700000000001, "2330": -0.192634, "2352": 0.01605, 
# "2357": -0.187006, "2382": -0.243372, "2395": -0.068415, "2408": -0.365457, "2412": -0.023018999999999998, "2454": -0.459061, "2474": -0.38257199999999997, 
# "2609": -0.131876, "2610": -0.27712, "2801": -0.15449400000000002, "2823": -0.347404, "2880": -0.17770399999999997, "2881": -0.46786900000000003, "2882": -0.25897400000000004,
# "2883": -0.31342, "2884": -0.19731400000000002, "2885": -0.344009, "2886": -0.245077, "2887": -0.064848, "2888": -0.21002400000000002, "2890": -0.24220300000000003,
# "2891": -0.368661, "2892": -0.17156, "2912": -0.05728900000000001, "3008": -0.367853, "3045": -0.086992, "4904": -0.142895, "4938": 0.001815, "5871": -0.174738, "5880": -0.186923,
#  "6505": 0.079374, "9904": -0.138743, "9910": -0.057804}

# #2016/05/10 2017/05/02 ,training model year = 2015to2016 , validation data year = 2016to2017
# dict_1617 = {'1101': 0.022023, '1102': -0.052175, '1216': -0.15324, '1301': -0.025627, '1326': -0.037925, '1402': -0.018943, '2002': 0.14674500000000001, 
#              '2105': -0.184696, '2207': -0.082674, '2301': -0.20282699999999998, '2303': -0.166446, '2308': 0.104733, '2317': 0.256694, '2327': 0.495589, 
#              '2330': 0.034511, '2352': 0.016662, '2357': -0.018402, '2382': -0.011867, '2395': -0.129885, '2408': 0.055124000000000006, '2412': -0.075364, 
#              '2454': -0.107002, '2474': 0.051997999999999996, '2609': -0.17872, '2610': 0.045202, '2801': -0.033925000000000004, '2823': -0.070153, '2880': -0.082949, 
#              '2881': -0.004619, '2882': 0.083315, '2883': 0.019838, '2884': 0.012423, '2885': -0.075406, '2886': -0.023999000000000003, '2887': -0.017715, 
#              '2888': 0.044952, '2890': -0.081944, '2891': 0.057888999999999996, '2892': -0.080136, '2912': -0.031194000000000003, '3008': 0.148872, 
#              '3045': -0.11116300000000001, '4904': -0.114104, '4938': 0.09371, '5871': -0.06908500000000001, '5880': 0.029977, '6505': -0.028698, '9904': -0.101618, '9910': 0.164909}

# #2017/05/09 2018/05/02 ,training model year = 2016to2017 , validation data year = 2017to2018
# dict_1718 = {'1101': 0.0886, '1102': 0.039432999999999996, '1216': 0.039179, '1301': -0.078937, '1326': 0.148837, '1402': 0.017634, '2002': -0.213798, '2105': -0.104019, 
#              '2207': -0.18028, '2301': -0.329351, '2303': -0.0694, '2308': -0.20213599999999998, '2317': -0.29909600000000003, '2327': 1.346812, '2330': -0.07529, 
#              '2352': -0.21352000000000002, '2357': -0.07881, '2382': -0.147031, '2395': -0.094841, '2408': 0.572618, '2412': -0.06770999999999999, '2454': -0.002265, 
#              '2474': 0.110378, '2609': -0.285927, '2610': 0.014276, '2801': -0.015103, '2823': 0.046685, '2880': -0.041138, '2881': -0.06826399999999999, '2882': 0.018207, 
#              '2883': 0.14633200000000002, '2884': -0.048371000000000004, '2885': -0.008484, '2886': -0.023857, '2887': -0.0021579999999999998, '2888': 0.023302999999999997, 
#              '2890': 0.0592, '2891': -0.0021390000000000003, '2892': -0.021456, '2912': -0.061447, '3008': -0.13997500000000002, '3045': -0.137306, '4904': -0.101847, 
#              '4938': -0.207732, '5871': 0.242905, '5880': 0.013685000000000001, '6505': -0.086952, '9904': -0.13780799999999999, '9910': -0.14646699999999999}

# #2018/05/09 2019/05/02 ,training model year = 2017to2018 , validation data year = 2018to2019 
# dict_1819 = {'1101': -0.10119600000000001, '1102': 0.30284300000000003, '1216': -0.178657, '1301': -0.150993, '1326': -0.206125, '1402': -0.018191, '2002': -0.085032, 
#              '2105': -0.194661, '2207': 0.5960540000000001, '2301': 0.002621, '2303': 0.066318, '2308': -0.08029299999999999, '2317': -0.10961800000000001, '2327': -0.441929,
#              '2330': 0.018359, '2352': -0.067327, '2357': -0.292248, '2382': 0.010492, '2395': 0.043683, '2408': -0.12920600000000002, '2412': -0.015959, 
#              '2454': -0.18540600000000002, '2474': -0.105421, '2609': -0.232895, '2610': 0.06848900000000001, '2801': -0.045191, '2823': -0.171992, '2880': 0.115928, 
#              '2881': -0.303297, '2882': -0.211085, '2883': -0.12726300000000001, '2884': 0.244944, '2885': 0.096821, '2886': 0.077208, '2887': -0.069478, 
#              '2888': 0.02754, '2890': 0.051294000000000006, '2891': -0.18335, '2892': -0.05027, '2912': -0.10264799999999999, '3008': 0.013419, '3045': -0.046723999999999995,
#              '4904': -0.08069000000000001, '4938': 0.042069, '5871': 0.09772600000000001, '5880': 0.163806, '6505': -0.081278, '9904': -0.11344599999999999, '9910': 0.043956}

# SequentialValROR = {'2013to2014': {"1101": 0.085478, "1102": -0.04748, "1216": -0.008218, "1301": 0.004213, "1326": 0.025005000000000003, "1402": -0.136284, "2002": -0.097525, "2105": -0.013768, "2207": 0.18027200000000002, "2301": -0.234631, "2303": 0.085366, "2308": 0.064972, "2317": 0.080144, "2327": 0.31522, "2330": 0.079063, "2352": 0.11302300000000001, "2357": 0.23427199999999998, "2382": 0.163342, "2395": 0.049181, "2408": -0.10364000000000001, "2412": -0.121255, "2454": 0.060306, "2474": 0.301863, "2609": -0.031976, "2610": -0.033377, "2801": -0.036392, "2823": -0.016068, "2880": -0.050206999999999995, "2881": -0.161019, "2882": -0.047247000000000004, "2883": -0.0030120000000000004, "2884": -0.141817, "2885": -0.073564, "2886": -0.099833, "2887": -0.126277, "2888": -0.13633699999999999, "2890": -0.137637, "2891": 0.00243, "2892": -0.031432, "2912": 0.18438400000000002, "3008": 0.11979799999999999, "3045": -0.10726200000000001, "4904": 0.01145, "4938": -0.233334, "5871": -0.18451599999999999, "5880": -0.089554, "6505": -0.07566200000000001, "9904": 0.060035, "9910": 0.453434},
#  '2014to2015': {"1101": -0.090756, "1102": -0.08095599999999999, "1216": 0.019461, "1301": -0.07437, "1326": -0.060762000000000004, "1402": -0.078458, "2002": -0.11341200000000001, "2105": -0.089865, "2207": 0.078741, "2301": -0.179477, "2303": -0.166447, "2308": -0.110653, "2317": -0.036009, "2327": 0.192796, "2330": 0.005849, "2352": -0.059463999999999996, "2357": -0.049455, "2382": -0.180788, "2395": 0.088559, "2408": -0.221508, "2412": -0.003836, "2454": -0.295076, "2474": 0.139645, "2609": 0.047446, "2610": 0.187228, "2801": -0.124141, "2823": -0.13999499999999998, "2880": 0.037312, "2881": 0.27954799999999996, "2882": 0.08104499999999999, "2883": 0.169998, "2884": -0.10841300000000001, "2885": -0.184097, "2886": -0.028828, "2887": -0.053741000000000004, "2888": -0.018932, "2890": 0.09290599999999999, "2891": 0.215363, "2892": 0.009198, "2912": -0.204211, "3008": 0.41324399999999994, "3045": -0.113139, "4904": -0.043006, "4938": 0.38023, "5871": 0.151407, "5880": -0.10687300000000001, "6505": -0.092741, "9904": 0.221375, "9910": 1.255873},
#  '2015to2016': {"1101": -0.143506, "1102": -0.160304, "1216": 0.033502000000000004, "1301": -0.106271, "1326": 0.09102, "1402": -0.073978, "2002": -0.045113, "2105": -0.089484, "2207": -0.295347, "2301": -0.088668, "2303": -0.163242, "2308": -0.393083, "2317": -0.207748, "2327": -0.20996700000000001, "2330": -0.192634, "2352": 0.01605, "2357": -0.187006, "2382": -0.243372, "2395": -0.068415, "2408": -0.365457, "2412": -0.023018999999999998, "2454": -0.459061, "2474": -0.38257199999999997, "2609": -0.131876, "2610": -0.27712, "2801": -0.15449400000000002, "2823": -0.347404, "2880": -0.17770399999999997, "2881": -0.46786900000000003, "2882": -0.25897400000000004, "2883": -0.31342, "2884": -0.19731400000000002, "2885": -0.344009, "2886": -0.245077, "2887": -0.064848, "2888": -0.21002400000000002, "2890": -0.24220300000000003, "2891": -0.368661, "2892": -0.17156, "2912": -0.05728900000000001, "3008": -0.367853, "3045": -0.086992, "4904": -0.142895, "4938": 0.001815, "5871": -0.174738, "5880": -0.186923, "6505": 0.079374, "9904": -0.138743, "9910": -0.057804},    
#  '2016to2017': {'1101': 0.022023, '1102': -0.052175, '1216': -0.15324, '1301': -0.025627, '1326': -0.037925, '1402': -0.018943, '2002': 0.14674500000000001, '2105': -0.184696, '2207': -0.082674, '2301': -0.20282699999999998, '2303': -0.166446, '2308': 0.104733, '2317': 0.256694, '2327': 0.495589, '2330': 0.034511, '2352': 0.016662, '2357': -0.018402, '2382': -0.011867, '2395': -0.129885, '2408': 0.055124000000000006, '2412': -0.075364, '2454': -0.107002, '2474': 0.051997999999999996, '2609': -0.17872, '2610': 0.045202, '2801': -0.033925000000000004, '2823': -0.070153, '2880': -0.082949, '2881': -0.004619, '2882': 0.083315, '2883': 0.019838, '2884': 0.012423, '2885': -0.075406, '2886': -0.023999000000000003, '2887': -0.017715, '2888': 0.044952, '2890': -0.081944, '2891': 0.057888999999999996, '2892': -0.080136, '2912': -0.031194000000000003, '3008': 0.148872, '3045': -0.11116300000000001, '4904': -0.114104, '4938': 0.09371, '5871': -0.06908500000000001, '5880': 0.029977, '6505': -0.028698, '9904': -0.101618, '9910': 0.164909}, 
#  '2017to2018': {'1101': 0.0886, '1102': 0.039432999999999996, '1216': 0.039179, '1301': -0.078937, '1326': 0.148837, '1402': 0.017634, '2002': -0.213798, '2105': -0.104019, '2207': -0.18028, '2301': -0.329351, '2303': -0.0694, '2308': -0.20213599999999998, '2317': -0.29909600000000003, '2327': 1.346812, '2330': -0.07529, '2352': -0.21352000000000002, '2357': -0.07881, '2382': -0.147031, '2395': -0.094841, '2408': 0.572618, '2412': -0.06770999999999999, '2454': -0.002265, '2474': 0.110378, '2609': -0.285927, '2610': 0.014276, '2801': -0.015103, '2823': 0.046685, '2880': -0.041138, '2881': -0.06826399999999999, '2882': 0.018207, '2883': 0.14633200000000002, '2884': -0.048371000000000004, '2885': -0.008484, '2886': -0.023857, '2887': -0.0021579999999999998, '2888': 0.023302999999999997, '2890': 0.0592, '2891': -0.0021390000000000003, '2892': -0.021456, '2912': -0.061447, '3008': -0.13997500000000002, '3045': -0.137306, '4904': -0.101847, '4938': -0.207732, '5871': 0.242905, '5880': 0.013685000000000001, '6505': -0.086952, '9904': -0.13780799999999999, '9910': -0.14646699999999999},
#  '2018to2019': {'1101': -0.10119600000000001, '1102': 0.30284300000000003, '1216': -0.178657, '1301': -0.150993, '1326': -0.206125, '1402': -0.018191, '2002': -0.085032, '2105': -0.194661, '2207': 0.5960540000000001, '2301': 0.002621, '2303': 0.066318, '2308': -0.08029299999999999, '2317': -0.10961800000000001, '2327': -0.441929, '2330': 0.018359, '2352': -0.067327, '2357': -0.292248, '2382': 0.010492, '2395': 0.043683, '2408': -0.12920600000000002, '2412': -0.015959, '2454': -0.18540600000000002, '2474': -0.105421, '2609': -0.232895, '2610': 0.06848900000000001, '2801': -0.045191, '2823': -0.171992, '2880': 0.115928, '2881': -0.303297, '2882': -0.211085, '2883': -0.12726300000000001, '2884': 0.244944, '2885': 0.096821, '2886': 0.077208, '2887': -0.069478, '2888': 0.02754, '2890': 0.051294000000000006, '2891': -0.18335, '2892': -0.05027, '2912': -0.10264799999999999, '3008': 0.013419, '3045': -0.046723999999999995, '4904': -0.08069000000000001, '4938': 0.042069, '5871': 0.09772600000000001, '5880': 0.163806, '6505': -0.081278, '9904': -0.11344599999999999, '9910': 0.043956}}

In [ ]:
# 3435 val ror
SequentialValROR = {'2013to2014': {'1101': 0.08632602040816321, '1102': -0.07716957142857146, '1216': -0.08528481632653065, '1301': 0.1965942653061224, '1326': -0.008374734693877596, '1402': 0.0002692448979591385, '2002': 0.0022561632653060773, '2105': -0.18283428571428575, '2207': -0.037161632653061265, '2301': -0.06114620408163269, '2303': -0.14038251020408166, '2308': 0.1489321632653061, '2317': 0.1337008571428571, '2327': 0.3755865714285714, '2330': -0.14150555102040818, '2352': 0.36576126530612235, '2357': 0.00096867346938771, '2382': -0.1467897755102041, '2395': 0.16198399999999996, '2408': -0.14112357142857146, '2412': -0.10028089795918371, '2454': 0.1270635714285714, '2474': 0.4167550204081632, '2609': -0.029538693877551064, '2610': -0.030901122448979638, '2801': -0.0782822040816327, '2823': -0.1383206530612245, '2880': -0.030135081632653105, '2881': -0.12289471428571432, '2882': -0.14516402040816329, '2883': -0.07599240816326534, '2884': -0.2115899591836735, '2885': -0.15000381632653065, '2886': -0.07378691836734698, '2887': 0.020771244897959135, '2888': -0.1415294897959184, '2890': 0.039457020408163214, '2891': 0.014791755102040772, '2892': -0.00481306122448984, '2912': -0.026929367346938817, '3008': 0.3300331632653061, '3045': -0.11719104081632656, '4904': -0.13793034693877554, '4938': 0.02798514285714281, '5871': -0.010979897959183717, '5880': -0.08224563265306126, '6505': -0.19479638775510208, '9904': 0.24199987755102037, '9910': 0.6337548163265305}, '2014to2015': {'1101': -0.08823553061224493, '1102': -0.057134897959183716, '1216': -0.06846834693877554, '1301': -0.010929938775510248, '1326': -0.07794081632653066, '1402': 0.04018038775510199, '2002': -0.03531834693877555, '2105': -0.1287180816326531, '2207': -0.07934279591836738, '2301': -0.132778306122449, '2303': -0.16184622448979596, '2308': -0.11131667346938778, '2317': -0.12648553061224493, '2327': 0.18627665306122446, '2330': 0.14299014285714282, '2352': -0.16025169387755106, '2357': -0.10845338775510208, '2382': -0.13224853061224492, '2395': 0.023401387755101997, '2408': -0.16481151020408166, '2412': 0.017518693877550975, '2454': -0.2853255510204082, '2474': 0.37935432653061213, '2609': 0.21352522448979588, '2610': -0.0014637142857143307, '2801': -0.08883920408163268, '2823': -0.13354538775510208, '2880': -0.0021704285714286165, '2881': 0.22850465306122444, '2882': -0.0451530204081633, '2883': 0.12427106122448976, '2884': -0.07535334693877555, '2885': -0.03950346938775515, '2886': -0.0661816734693878, '2887': -0.034889530612244944, '2888': -0.0008319387755102492, '2890': 0.028606510204081587, '2891': 0.1109382040816326, '2892': 0.02851283673469383, '2912': -0.2135654285714286, '3008': 0.29897624489795915, '3045': -0.09205636734693881, '4904': 0.1895458571428571, '4938': -0.041092795918367384, '5871': 0.15358877551020403, '5880': -0.012215346938775556, '6505': -0.045563102040816365, '9904': 0.1600210204081632, '9910': 1.2412647346938777}, '2015to2016': {'1101': -0.09569922448979595, '1102': -0.1669753673469388, '1216': -0.06527928571428575, '1301': -0.05022595918367351, '1326': 0.0188863265306122, '1402': -0.056950673469387786, '2002': -0.03177853061224494, '2105': 0.020786857142857097, '2207': -0.34693667346938784, '2301': -0.14091957142857145, '2303': -0.14912120408163268, '2308': -0.34504551020408164, '2317': -0.11418412244897963, '2327': -0.1443854897959184, '2330': -0.10552557142857147, '2352': -0.28260381632653064, '2357': -0.26142424489795923, '2382': -0.09522148979591841, '2395': -0.031125938775510245, '2408': -0.1188584285714286, '2412': -0.01654410204081637, '2454': -0.050689122448979634, '2474': -0.32265963265306125, '2609': -0.3135014897959184, '2610': -0.05195163265306126, '2801': -0.08298669387755106, '2823': -0.3290304693877551, '2880': -0.1665163673469388, '2881': -0.466249612244898, '2882': -0.2919139183673469, '2883': -0.24820275510204085, '2884': 0.06463332653061221, '2885': -0.36360118367346944, '2886': -0.3399538367346939, '2887': -0.07214034693877554, '2888': -0.21174191836734696, '2890': -0.15462087755102044, '2891': -0.3626852653061225, '2892': -0.18760538775510205, '2912': -0.014545734693877594, '3008': -0.1292832448979592, '3045': -0.10796836734693882, '4904': -0.16374779591836738, '4938': -0.44276567346938783, '5871': -0.17350232653061226, '5880': -0.21285246938775515, '6505': 0.1955097346938775, '9904': -0.1177603673469388, '9910': -0.09778397959183677}, '2016to2017': {'1101': -0.08817412244897962, '1102': -0.06326738775510207, '1216': -0.08963751020408166, '1301': -0.04291838775510208, '1326': 0.07239053061224485, '1402': -0.06083395918367351, '2002': 0.08709934693877547, '2105': -0.1638092040816327, '2207': -0.14385675510204085, '2301': -0.20768169387755106, '2303': -0.13849759183673474, '2308': 0.14991365306122445, '2317': 0.2206423265306122, '2327': 0.4715415102040815, '2330': 0.031823673469387706, '2352': 0.033098673469387704, '2357': -0.037692448979591876, '2382': -0.045417387755102075, '2395': -0.06641689795918371, '2408': 0.0048654897959183215, '2412': -0.09345210204081637, '2454': -0.0876224897959184, '2474': 0.16101187755102037, '2609': 0.18317710204081628, '2610': -0.001780122448979637, '2801': -0.02666291836734698, '2823': -0.12699865306122451, '2880': 0.0732658571428571, '2881': -0.009821469387755147, '2882': 0.18542422448979587, '2883': -0.08069689795918372, '2884': -0.04084924489795923, '2885': 0.09396873469387751, '2886': -0.10340334693877555, '2887': -0.005188795918367391, '2888': 0.0008208775510203629, '2890': -0.0768916734693878, '2891': 0.08638326530612239, '2892': -0.12622220408163268, '2912': -0.08682210204081636, '3008': 0.2381561428571428, '3045': -0.1038134285714286, '4904': -0.11137600000000004, '4938': -0.01808763265306127, '5871': 0.036252346938775465, '5880': 0.010537938775510158, '6505': 0.04347561224489791, '9904': 0.05146179591836731, '9910': -0.061029632653061265}, '2017to2018': {'1101': -0.024829000000000042, '1102': -0.1353584897959184, '1216': -0.08617159183673474, '1301': 0.03347753061224485, '1326': 0.008250224489795873, '1402': -0.1239240816326531, '2002': -0.10644877551020412, '2105': -0.21031079591836735, '2207': -0.12132204081632655, '2301': -0.25023234693877555, '2303': -0.11991693877551024, '2308': -0.17082118367346943, '2317': -0.40638914285714295, '2327': 1.2444350612244899, '2330': -0.11412375510204086, '2352': -0.307488693877551, '2357': -0.0019341632653061675, '2382': -0.32680104081632655, '2395': -0.10021324489795923, '2408': 0.3559526122448979, '2412': -0.054845102040816364, '2454': 0.1470389183673469, '2474': -0.08618928571428576, '2609': -0.2443798367346939, '2610': 0.10209126530612239, '2801': -0.10426930612244903, '2823': 0.03381891836734689, '2880': -0.037930795918367385, '2881': -0.0008007142857143308, '2882': 0.002367530612244853, '2883': 0.048541265306122405, '2884': -0.05789469387755106, '2885': -0.06202465306122453, '2886': -0.03135179591836739, '2887': 0.02978367346938771, '2888': -0.0668956734693878, '2890': 0.06767251020408159, '2891': -0.13350375510204085, '2892': -0.05881685714285718, '2912': -0.02990818367346943, '3008': 0.05013579591836731, '3045': -0.15624559183673473, '4904': -0.07957697959183677, '4938': -0.11151546938775514, '5871': 0.16498363265306118, '5880': -0.15040453061224493, '6505': -0.07111306122448983, '9904': -0.1695482653061225, '9910': -0.3188471224489796}, '2018to2019': {'1101': -0.1158421428571429, '1102': 0.1406233265306122, '1216': -0.2446119387755102, '1301': -0.07228918367346943, '1326': -0.22041400000000003, '1402': -0.17592326530612248, '2002': -0.08724363265306126, '2105': -0.14228824489795922, '2207': 0.8514072040816325, '2301': -0.13765244897959186, '2303': 0.025158285714285668, '2308': -0.10370622448979595, '2317': -0.28447, '2327': -0.37603165306122455, '2330': 0.14937659183673463, '2352': -0.06054357142857147, '2357': -0.05421124489795922, '2382': 0.04232030612244893, '2395': -0.02803575510204086, '2408': 0.15409253061224484, '2412': -0.07206644897959187, '2454': 0.008177367346938729, '2474': -0.08246628571428576, '2609': -0.08936169387755105, '2610': -0.07173546938775514, '2801': -0.05182257142857147, '2823': -0.21767873469387758, '2880': 0.1386884489795918, '2881': -0.19262316326530615, '2882': -0.17827342857142858, '2883': -0.18475563265306125, '2884': 0.15211289795918362, '2885': 0.152296081632653, '2886': -0.017553693877551065, '2887': -0.01157836734693882, '2888': -0.011570040816326574, '2890': -0.04679022448979595, '2891': -0.09985624489795923, '2892': -0.034300428571428614, '2912': -0.2577553673469388, '3008': -0.018405081632653107, '3045': -0.08053453061224493, '4904': -0.03665579591836739, '4938': 0.02445157142857138, '5871': 0.08168189795918362, '5880': 0.07513412244897955, '6505': -0.12422383673469391, '9904': -0.10908932653061228, '9910': 0.2941197959183673}}
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
validation_years = ["2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018", "2018to2019"]
# make the dictionary of validation asset allocation per year
SequentialTestWeight = {}
for validation_year in validation_years:
  print(validation_year)
  Weights_Dict = {}
  for stock_num in stockNum:
    if SequentialValROR[validation_year][stock_num] > 0: 
      Weights_Dict[stock_num] = SequentialValROR[validation_year][stock_num]
    else: Weights_Dict[stock_num] = 0
  WeightSum = sum(Weights_Dict.values())
  for stock_num in stockNum:
    Weights_Dict[stock_num] /= WeightSum
  SequentialTestWeight[validation_year] = Weights_Dict
  print(SequentialTestWeight[validation_year])
print(SequentialTestWeight)

2013to2014
{'1101': 0.025962784454750462, '1102': 0.0, '1216': 0.0, '1301': 0.05912625777314556, '1326': 0.0, '1402': 8.097613232027745e-05, '2002': 0.0006785472129366054, '2105': 0.0, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.04479175148992739, '2317': 0.040210894919084354, '2327': 0.1129586786462895, '2330': 0.0, '2352': 0.11000369121778339, '2357': 0.0002913311696037622, '2382': 0.0, '2395': 0.04871712673923527, '2408': 0.0, '2412': 0.0, '2454': 0.03821471326319629, '2474': 0.12534020118306174, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.0, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.0, '2885': 0.0, '2886': 0.0, '2887': 0.006247008162692423, '2888': 0.0, '2890': 0.011866805758451948, '2891': 0.004448660410916204, '2892': 0.0, '2912': 0.0, '3008': 0.09925836775821467, '3045': 0.0, '4904': 0.0, '4938': 0.00841660751300765, '5871': 0.0, '5880': 0.0, '6505': 0.0, '9904': 0.07278211863846108, '9910': 0.19060347755692147}
2014to2015
{'1101': 0.0, '1102': 0.0,

In [ ]:
# define the dictionary of validation rate of return per year
# SequentialValROR = {'2013to2014': {"1101": 0.085478, "1102": -0.04748, "1216": -0.008218, "1301": 0.004213, "1326": 0.025005000000000003, "1402": -0.136284, "2002": -0.097525, "2105": -0.013768, "2207": 0.18027200000000002, "2301": -0.234631, "2303": 0.085366, "2308": 0.064972, "2317": 0.080144, "2327": 0.31522, "2330": 0.079063, "2352": 0.11302300000000001, "2357": 0.23427199999999998, "2382": 0.163342, "2395": 0.049181, "2408": -0.10364000000000001, "2412": -0.121255, "2454": 0.060306, "2474": 0.301863, "2609": -0.031976, "2610": -0.033377, "2801": -0.036392, "2823": -0.016068, "2880": -0.050206999999999995, "2881": -0.161019, "2882": -0.047247000000000004, "2883": -0.0030120000000000004, "2884": -0.141817, "2885": -0.073564, "2886": -0.099833, "2887": -0.126277, "2888": -0.13633699999999999, "2890": -0.137637, "2891": 0.00243, "2892": -0.031432, "2912": 0.18438400000000002, "3008": 0.11979799999999999, "3045": -0.10726200000000001, "4904": 0.01145, "4938": -0.233334, "5871": -0.18451599999999999, "5880": -0.089554, "6505": -0.07566200000000001, "9904": 0.060035, "9910": 0.453434},
#  '2014to2015': {"1101": -0.090756, "1102": -0.08095599999999999, "1216": 0.019461, "1301": -0.07437, "1326": -0.060762000000000004, "1402": -0.078458, "2002": -0.11341200000000001, "2105": -0.089865, "2207": 0.078741, "2301": -0.179477, "2303": -0.166447, "2308": -0.110653, "2317": -0.036009, "2327": 0.192796, "2330": 0.005849, "2352": -0.059463999999999996, "2357": -0.049455, "2382": -0.180788, "2395": 0.088559, "2408": -0.221508, "2412": -0.003836, "2454": -0.295076, "2474": 0.139645, "2609": 0.047446, "2610": 0.187228, "2801": -0.124141, "2823": -0.13999499999999998, "2880": 0.037312, "2881": 0.27954799999999996, "2882": 0.08104499999999999, "2883": 0.169998, "2884": -0.10841300000000001, "2885": -0.184097, "2886": -0.028828, "2887": -0.053741000000000004, "2888": -0.018932, "2890": 0.09290599999999999, "2891": 0.215363, "2892": 0.009198, "2912": -0.204211, "3008": 0.41324399999999994, "3045": -0.113139, "4904": -0.043006, "4938": 0.38023, "5871": 0.151407, "5880": -0.10687300000000001, "6505": -0.092741, "9904": 0.221375, "9910": 1.255873},
#  '2015to2016': {"1101": -0.143506, "1102": -0.160304, "1216": 0.033502000000000004, "1301": -0.106271, "1326": 0.09102, "1402": -0.073978, "2002": -0.045113, "2105": -0.089484, "2207": -0.295347, "2301": -0.088668, "2303": -0.163242, "2308": -0.393083, "2317": -0.207748, "2327": -0.20996700000000001, "2330": -0.192634, "2352": 0.01605, "2357": -0.187006, "2382": -0.243372, "2395": -0.068415, "2408": -0.365457, "2412": -0.023018999999999998, "2454": -0.459061, "2474": -0.38257199999999997, "2609": -0.131876, "2610": -0.27712, "2801": -0.15449400000000002, "2823": -0.347404, "2880": -0.17770399999999997, "2881": -0.46786900000000003, "2882": -0.25897400000000004, "2883": -0.31342, "2884": -0.19731400000000002, "2885": -0.344009, "2886": -0.245077, "2887": -0.064848, "2888": -0.21002400000000002, "2890": -0.24220300000000003, "2891": -0.368661, "2892": -0.17156, "2912": -0.05728900000000001, "3008": -0.367853, "3045": -0.086992, "4904": -0.142895, "4938": 0.001815, "5871": -0.174738, "5880": -0.186923, "6505": 0.079374, "9904": -0.138743, "9910": -0.057804},    
#  '2016to2017': {'1101': 0.022023, '1102': -0.052175, '1216': -0.15324, '1301': -0.025627, '1326': -0.037925, '1402': -0.018943, '2002': 0.14674500000000001, '2105': -0.184696, '2207': -0.082674, '2301': -0.20282699999999998, '2303': -0.166446, '2308': 0.104733, '2317': 0.256694, '2327': 0.495589, '2330': 0.034511, '2352': 0.016662, '2357': -0.018402, '2382': -0.011867, '2395': -0.129885, '2408': 0.055124000000000006, '2412': -0.075364, '2454': -0.107002, '2474': 0.051997999999999996, '2609': -0.17872, '2610': 0.045202, '2801': -0.033925000000000004, '2823': -0.070153, '2880': -0.082949, '2881': -0.004619, '2882': 0.083315, '2883': 0.019838, '2884': 0.012423, '2885': -0.075406, '2886': -0.023999000000000003, '2887': -0.017715, '2888': 0.044952, '2890': -0.081944, '2891': 0.057888999999999996, '2892': -0.080136, '2912': -0.031194000000000003, '3008': 0.148872, '3045': -0.11116300000000001, '4904': -0.114104, '4938': 0.09371, '5871': -0.06908500000000001, '5880': 0.029977, '6505': -0.028698, '9904': -0.101618, '9910': 0.164909}, 
#  '2017to2018': {'1101': 0.0886, '1102': 0.039432999999999996, '1216': 0.039179, '1301': -0.078937, '1326': 0.148837, '1402': 0.017634, '2002': -0.213798, '2105': -0.104019, '2207': -0.18028, '2301': -0.329351, '2303': -0.0694, '2308': -0.20213599999999998, '2317': -0.29909600000000003, '2327': 1.346812, '2330': -0.07529, '2352': -0.21352000000000002, '2357': -0.07881, '2382': -0.147031, '2395': -0.094841, '2408': 0.572618, '2412': -0.06770999999999999, '2454': -0.002265, '2474': 0.110378, '2609': -0.285927, '2610': 0.014276, '2801': -0.015103, '2823': 0.046685, '2880': -0.041138, '2881': -0.06826399999999999, '2882': 0.018207, '2883': 0.14633200000000002, '2884': -0.048371000000000004, '2885': -0.008484, '2886': -0.023857, '2887': -0.0021579999999999998, '2888': 0.023302999999999997, '2890': 0.0592, '2891': -0.0021390000000000003, '2892': -0.021456, '2912': -0.061447, '3008': -0.13997500000000002, '3045': -0.137306, '4904': -0.101847, '4938': -0.207732, '5871': 0.242905, '5880': 0.013685000000000001, '6505': -0.086952, '9904': -0.13780799999999999, '9910': -0.14646699999999999},
#  '2018to2019': {'1101': -0.10119600000000001, '1102': 0.30284300000000003, '1216': -0.178657, '1301': -0.150993, '1326': -0.206125, '1402': -0.018191, '2002': -0.085032, '2105': -0.194661, '2207': 0.5960540000000001, '2301': 0.002621, '2303': 0.066318, '2308': -0.08029299999999999, '2317': -0.10961800000000001, '2327': -0.441929, '2330': 0.018359, '2352': -0.067327, '2357': -0.292248, '2382': 0.010492, '2395': 0.043683, '2408': -0.12920600000000002, '2412': -0.015959, '2454': -0.18540600000000002, '2474': -0.105421, '2609': -0.232895, '2610': 0.06848900000000001, '2801': -0.045191, '2823': -0.171992, '2880': 0.115928, '2881': -0.303297, '2882': -0.211085, '2883': -0.12726300000000001, '2884': 0.244944, '2885': 0.096821, '2886': 0.077208, '2887': -0.069478, '2888': 0.02754, '2890': 0.051294000000000006, '2891': -0.18335, '2892': -0.05027, '2912': -0.10264799999999999, '3008': 0.013419, '3045': -0.046723999999999995, '4904': -0.08069000000000001, '4938': 0.042069, '5871': 0.09772600000000001, '5880': 0.163806, '6505': -0.081278, '9904': -0.11344599999999999, '9910': 0.043956}}

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
validation_years = ["2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018", "2018to2019"]
# make the dictionary of validation asset allocation per year
SequentialWeights_Dict = {}
for validation_year in validation_years:
  print(validation_year)
  Weights_Dict = {}
  for stock_num in stockNum:
    if SequentialValROR[validation_year][stock_num] > 0: Weights_Dict[stock_num] = SequentialValROR[validation_year][stock_num]
    else: Weights_Dict[stock_num] = 0
  WeightSumThisYear = sum(Weights_Dict.values())
  for stock_num in stockNum:
    Weights_Dict[stock_num] /= WeightSumThisYear
  SequentialWeights_Dict[validation_year] = Weights_Dict
  print(sum(Weights_Dict.values()))
print(SequentialWeights_Dict)
# calculate the sum of total weights
TotalWeights = 0
for validation_year in validation_years:
  TotalWeights += sum(SequentialWeights_Dict[validation_year].values())
print(TotalWeights)
# calculate the final testing weights
TestingWeights = {}
for stock_num in stockNum:
  assetAllocationThisStock = 0
  for validation_year in validation_years:
    assetAllocationThisStock += SequentialWeights_Dict[validation_year][stock_num]
  TestingWeights[stock_num] = assetAllocationThisStock / TotalWeights
print(sum(TestingWeights.values()))
print(TestingWeights)

2013to2014
1.0000000000000002
2014to2015
1.0
2015to2016
1.0
2016to2017
1.0000000000000002
2017to2018
1.0000000000000002
2018to2019
0.9999999999999999
{'2013to2014': {'1101': 0.025962784454750462, '1102': 0.0, '1216': 0.0, '1301': 0.05912625777314556, '1326': 0.0, '1402': 8.097613232027745e-05, '2002': 0.0006785472129366054, '2105': 0.0, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.04479175148992739, '2317': 0.040210894919084354, '2327': 0.1129586786462895, '2330': 0.0, '2352': 0.11000369121778339, '2357': 0.0002913311696037622, '2382': 0.0, '2395': 0.04871712673923527, '2408': 0.0, '2412': 0.0, '2454': 0.03821471326319629, '2474': 0.12534020118306174, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.0, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.0, '2885': 0.0, '2886': 0.0, '2887': 0.006247008162692423, '2888': 0.0, '2890': 0.011866805758451948, '2891': 0.004448660410916204, '2892': 0.0, '2912': 0.0, '3008': 0.09925836775821467, '3045': 0.0, '4904': 0.0, '4938': 0

In [ ]:
#測試資料集的投資組合分佈
"""
這些是預測35天漲跌30天買賣的
TestingWeights = {'1101': 0.01231937568269382, '1102': 0.026469216124037975, '1216': 0.028206301545257648, '1301': 0.0002626639498747655, '1326': 0.07843775032269733, '1402': 0.0010037280351246753, '2002': 0.012973658553145985, '2105': 0.0, '2207': 0.06214480052060492, '2301': 0.00020965618305760468, '2303': 0.010627071110092381, '2308': 0.013310144981576457, '2317': 0.02769086213595706, '2327': 0.14802851191139688, '2330': 0.00968860174134243, '2352': 0.02058215433240366, '2357': 0.014605936117982686, '2382': 0.011022994756957878, '2395': 0.01018945448605091, '2408': 0.03746692879124049, '2412': 0.0, '2454': 0.0037598414814022335, '2474': 0.0354221800985154, '2609': 0.0019442417399844873, '2610': 0.017959603603768543, '2801': 0.0, '2823': 0.0026573121991491136, '2880': 0.010802157349682305, '2881': 0.01145531530432878, '2882': 0.011723245633454173, '2883': 0.017049268681434044, '2884': 0.020691605968487816, '2885': 0.007744800190698338, '2886': 0.006175938413396239, '2887': 0.0, '2888': 0.007503543330459228, '2890': 0.011279821618867333, '2891': 0.01409458400574091, '2892': 0.0003769155571465943, '2912': 0.011495615887421972, '3008': 0.0386379457155125, '3045': 0.0, '4904': 0.0007138623845397733, '4938': 0.028595138600695194, '5871': 0.02784770958500966, '5880': 0.01653219519052777, '6505': 0.05965431252564698, '9904': 0.01281444835883596, '9910': 0.09782858529379906}
"""
TestingWeights = {'1101': 0.00432713074245841, '1102': 0.01023620163060211, '1216': 0.0, '1301': 0.012292422666671346, '1326': 0.016497166928505978, '1402': 0.0018906579011205651, '2002': 0.006607291765596248, '2105': 0.01155533180550407, '2207': 0.0619753210633201, '2301': 0.0, '2303': 0.0018313127103821158, '2308': 0.018642979973452028, '2317': 0.023153093259172725, '2327': 0.1533151817263302, '2330': 0.01992643114953589, '2352': 0.020801813468110244, '2357': 4.855519493396037e-05, '2382': 0.003080564208128652, '2395': 0.009212795615201521, '2408': 0.03750215163278853, '2412': 0.0008184446730186342, '2454': 0.017672670578195875, '2474': 0.05061802709250079, '2609': 0.02363338741115363, '2610': 0.007434934248536427, '2801': 0.0, '2823': 0.0024629083953842047, '2880': 0.01555812266273388, '2881': 0.010675362978833815, '2882': 0.01399780474434732, '2883': 0.009340823548907487, '2884': 0.047001957305216585, '2885': 0.018092268432637098, '2886': 0.0, '2887': 0.003210204264062667, '2888': 6.12053206104872e-05, '2890': 0.008242592176790004, '2891': 0.012365103435358307, '2892': 0.001332073013049445, '2912': 0.0, '3008': 0.051919070922829214, '3045': 0.0, '4904': 0.008855271868761555, '4938': 0.0031826377420210244, '5871': 0.02783933600389173, '5880': 0.006254853074356816, '6505': 0.11192466772958853, '9904': 0.023443307766394042, '9910': 0.11116656116900576}
print(TestingWeights)

{'1101': 0.00432713074245841, '1102': 0.01023620163060211, '1216': 0.0, '1301': 0.012292422666671346, '1326': 0.016497166928505978, '1402': 0.0018906579011205651, '2002': 0.006607291765596248, '2105': 0.01155533180550407, '2207': 0.0619753210633201, '2301': 0.0, '2303': 0.0018313127103821158, '2308': 0.018642979973452028, '2317': 0.023153093259172725, '2327': 0.1533151817263302, '2330': 0.01992643114953589, '2352': 0.020801813468110244, '2357': 4.855519493396037e-05, '2382': 0.003080564208128652, '2395': 0.009212795615201521, '2408': 0.03750215163278853, '2412': 0.0008184446730186342, '2454': 0.017672670578195875, '2474': 0.05061802709250079, '2609': 0.02363338741115363, '2610': 0.007434934248536427, '2801': 0.0, '2823': 0.0024629083953842047, '2880': 0.01555812266273388, '2881': 0.010675362978833815, '2882': 0.01399780474434732, '2883': 0.009340823548907487, '2884': 0.047001957305216585, '2885': 0.018092268432637098, '2886': 0.0, '2887': 0.003210204264062667, '2888': 6.12053206104872e

Start calculating the Rate of Return of Testing Set 開始計算測試資料集的投資報酬率

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# start = '2019/05/09'
# end = '2020/12/02'
start = '2019/06/04'
end = '2020/06/04'
# end = '2020/09/21'
# end = '2020/12/01'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))
asset_at_the_start = 20 * 10**6
total_asset_present_value = 0
ROR_Test = {}
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2012to2013.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2013to2014.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2014to2015.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2015to2016.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2016to2017.h5')
model6 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2017to2018.h5')
for stock_num in stockNum:
  if TestingWeights[stock_num] == 0.0: continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 5
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  Return_Test_Stock_df.reset_index()
  Return_Test_Stock_df = Return_Test_Stock_df[30:]
  # start evaluaion ###############################################
  y_preds_list=[]
  y_truth_list=[]
  #if TestingWeights[stock_num] == 0.0: continue
  left_money = asset_at_the_start * TestingWeights[stock_num] #10**6
  own_asset = left_money 
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_hold_stock = 0 
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    test_x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model6.predict(test_x.reshape(1,30,23)))])
    if predict >= 3 : predict = 1
    else: predict = 0
    y_truth_list.append(new_df['y_label'][i])
    y_preds_list.append(predict)
    price = int(Price_df['Close'][i]*100)/100
    if predict == 1:
      if own_stock == 0:
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
    elif predict == 0:
      if own_stock > 0:
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
  ROR_Test[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
  print(stock_num)
  print(ROR_Test[stock_num])
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")

In [ ]:
ROR_Test

{'1101': -6.766377826108991e-07,
 '1102': -4.549588950774224e-07,
 '1216': -5.612565655646025e-07,
 '1301': -4.14546810891745e-05,
 '1326': -1.9925124965626222e-07,
 '1402': -1.3698342487600012e-05,
 '2002': -4.23313244506143e-07,
 '2207': -7.405667952096268e-08,
 '2301': -1.4890250644459216e-05,
 '2303': -9.302283275535801e-07,
 '2308': -1.5961010217810597e-07,
 '2317': -1.8031579855982387e-07,
 '2327': -1.1533691812667788e-08,
 '2330': -1.0222948574513352e-06,
 '2352': -5.575344843184499e-07,
 '2357': -1.2153028470416498e-06,
 '2382': -1.3757714193715764e-06,
 '2395': -5.404076421471244e-07,
 '2408': -2.782158530713546e-07,
 '2454': -1.2624756600963987e-06,
 '2474': -4.752166364067008e-07,
 '2609': -8.872450485076963e-06,
 '2610': -6.551945192272683e-07,
 '2823': -2.28678441921199e-06,
 '2880': -1.3416338033080895e-06,
 '2881': -8.015388102922495e-07,
 '2882': -6.27381710689379e-08,
 '2883': -1.979739811136019e-07,
 '2884': -6.829703379482196e-07,
 '2885': -5.516386330805582e-07,
 '2

35漲34進-Ensemble

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
start = '2019/06/04'
# end = '2020/06/04'
# end = '2020/09/21'
end = '2020/12/01'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))
asset_at_the_start = 20 * 10**6
total_asset_present_value = 0
ROR_Test = {}
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2012to2013.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2013to2014.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2014to2015.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2015to2016.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2016to2017.h5')
model6 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2017to2018.h5')
for stock_num in stockNum:
  if TestingWeights[stock_num] == 0.0: continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 5
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  Return_Test_Stock_df.reset_index()
  Return_Test_Stock_df = Return_Test_Stock_df[30:]
  # start evaluaion ###############################################
  time_slide = 30 
  y_preds_list=[0]*(time_slide -1 )
  # y_truth_list=[]
  #if TestingWeights[stock_num] == 0.0: continue
  left_money = asset_at_the_start * TestingWeights[stock_num] #10**6
  own_asset = left_money 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_hold_stock = 0 
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    test_x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model6.predict(test_x.reshape(1,30,23)))])
    if predict >= 3 : predict = 1
    else: predict = 0
    # y_truth_list.append(new_df['y_label'][i])
    y_preds_list.append(predict)
  for i in range(time_slide, len(new_df)): 
    price = int(Price_df['Close'][i]*100)/100
    if y_preds_list[i-4] == 1:
      if own_stock == 0:
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
    elif y_preds_list[i-4] == 0:
      if own_stock > 0:
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
    money_record.append(left_money+own_stock*price)
  #print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
  ROR_Test[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
  print(stock_num)
  print(ROR_Test[stock_num])
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")
print(ROR_Test)

1101
-0.062427219914538125
1102
-0.1693745095269414
1301
-0.07176963326060458
1326
-0.13062951583415555
1402
-0.11358897926128331
2002
-0.1998083045871234
2105
-0.013018389089650861
2207
1.0171150040880965
2303
0.601801802264522
2308
0.8487924167210223
2317
-0.17223155517645103
2327
0.396561303251733
2330
0.37119385779406805
2352
0.419866592173764
2357
0.033874955466180676
2382
0.083454125446559
2395
0.027641380036510135
2408
-0.10968841662919429
2412
-0.08576593547825068
2454
0.4336854120542629
2474
-0.3639163387154193
2609
0.4773760270828503
2610
0.10710461247459283
2823
0.05294212519644061
2880
-0.07583644172956737
2881
0.00673391821043617
2882
-0.20068537857564642
2883
-0.20497909406836284
2884
0.057797437607601705
2885
-0.059755825349509
2887
-0.22355408099621712
2888
-0.10056839093548535
2890
-0.27790919745371795
2891
-0.10855982259879977
2892
-0.13837305556358656
3008
-0.3025674505265816
4904
-0.20425368024465992
4938
-0.04599572866501163
5871
0.1557711001261687
5880
-0.23305152

35漲34進-各個年度

In [ ]:
SequentialTestWeight = {'2013to2014': {'1101': 0.025962784454750462, '1102': 0.0, '1216': 0.0, '1301': 0.05912625777314556, '1326': 0.0, '1402': 8.097613232027745e-05, '2002': 0.0006785472129366054, '2105': 0.0, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.04479175148992739, '2317': 0.040210894919084354, '2327': 0.1129586786462895, '2330': 0.0, '2352': 0.11000369121778339, '2357': 0.0002913311696037622, '2382': 0.0, '2395': 0.04871712673923527, '2408': 0.0, '2412': 0.0, '2454': 0.03821471326319629, '2474': 0.12534020118306174, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.0, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.0, '2885': 0.0, '2886': 0.0, '2887': 0.006247008162692423, '2888': 0.0, '2890': 0.011866805758451948, '2891': 0.004448660410916204, '2892': 0.0, '2912': 0.0, '3008': 0.09925836775821467, '3045': 0.0, '4904': 0.0, '4938': 0.00841660751300765, '5871': 0.0, '5880': 0.0, '6505': 0.0, '9904': 0.07278211863846108, '9910': 0.19060347755692147}, '2014to2015': {'1101': 0.0, '1102': 0.0, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.011262971274403113, '2002': 0.0, '2105': 0.0, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.05221524006457912, '2330': 0.040081591082164233, '2352': 0.0, '2357': 0.0, '2382': 0.0, '2395': 0.006559646951973859, '2408': 0.0, '2412': 0.004910668038111805, '2454': 0.0, '2474': 0.10633687530783706, '2609': 0.05985329172149854, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.0, '2881': 0.0640521778730029, '2882': 0.0, '2883': 0.03483444214978472, '2884': 0.0, '2885': 0.0, '2886': 0.0, '2887': 0.0, '2888': 0.0, '2890': 0.008018695704313584, '2891': 0.031097106713377624, '2892': 0.00799243807829667, '2912': 0.0, '3008': 0.08380608167692574, '3045': 0.0, '4904': 0.053131631212569336, '4938': 0.0, '5871': 0.04305249559027769, '5880': 0.0, '6505': 0.0, '9904': 0.04485551924343896, '9910': 0.34793912731744514}, '2015to2016': {'1101': 0.0, '1102': 0.0, '1216': 0.0, '1301': 0.0, '1326': 0.06299300605622643, '1402': 0.0, '2002': 0.0, '2105': 0.06933199083302442, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.0, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.0, '2395': 0.0, '2408': 0.0, '2412': 0.0, '2454': 0.0, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.0, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.21557646602041144, '2885': 0.0, '2886': 0.0, '2887': 0.0, '2888': 0.0, '2890': 0.0, '2891': 0.0, '2892': 0.0, '2912': 0.0, '3008': 0.0, '3045': 0.0, '4904': 0.0, '4938': 0.0, '5871': 0.0, '5880': 0.0, '6505': 0.6520985370903377, '9904': 0.0, '9910': 0.0}, '2016to2017': {'1101': 0.0, '1102': 0.0, '1216': 0.0, '1301': 0.0, '1326': 0.03238499308291469, '1402': 0.0, '2002': 0.038965203380640884, '2105': 0.0, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.06706612835078476, '2317': 0.09870766463595199, '2327': 0.21095118957013503, '2330': 0.014236799156777133, '2352': 0.014807189590878077, '2357': 0.0, '2382': 0.0, '2395': 0.0, '2408': 0.002176650068084376, '2412': 0.0, '2454': 0.0, '2474': 0.07203108606410591, '2609': 0.08194703274542325, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.03277658357685665, '2881': 0.0, '2882': 0.08295231678397343, '2883': 0.0, '2884': 0.0, '2885': 0.04203832735212354, '2886': 0.0, '2887': 0.0, '2888': 0.0003672319236629232, '2890': 0.0, '2891': 0.03864485348785602, '2892': 0.0, '2912': 0.0, '3008': 0.10654273388870025, '3045': 0.0, '4904': 0.0, '4938': 0.0, '5871': 0.016218032868695116, '5880': 0.004714305468778385, '6505': 0.01944946928719354, '9904': 0.023022208716464213, '9910': 0.0}, '2017to2018': {'1101': 0.0, '1102': 0.0, '1216': 0.0, '1301': 0.014628278226882504, '1326': 0.0036050024318947547, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.0, '2301': 0.0, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.5437659820769776, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.0, '2395': 0.0, '2408': 0.15553637775181275, '2412': 0.0, '2454': 0.06424984665000007, '2474': 0.0, '2609': 0.0, '2610': 0.04460960549121856, '2801': 0.0, '2823': 0.014777450372305228, '2880': 0.0, '2881': 0.0, '2882': 0.0010345116821105004, '2883': 0.0212104991436602, '2884': 0.0, '2885': 0.0, '2886': 0.0, '2887': 0.01301421742168358, '2888': 0.0, '2890': 0.029570051597974493, '2891': 0.0, '2892': 0.0, '2912': 0.0, '3008': 0.021907242213134632, '3045': 0.0, '4904': 0.0, '4938': 0.0, '5871': 0.07209093494034513, '5880': 0.0, '6505': 0.0, '9904': 0.0, '9910': 0.0}, '2018to2019': {'1101': 0.0, '1102': 0.06141720978361267, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.3718519263799206, '2301': 0.0, '2303': 0.010987876262292695, '2308': 0.0, '2317': 0.0, '2327': 0.0, '2330': 0.06524019665827396, '2352': 0.0, '2357': 0.0, '2382': 0.01848338524877191, '2395': 0.0, '2408': 0.06729988197683405, '2412': 0.0, '2454': 0.0035714635559788844, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.06057215239954663, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.06643527781088807, '2885': 0.06651528324369904, '2886': 0.0, '2887': 0.0, '2888': 0.0, '2890': 0.0, '2891': 0.0, '2892': 0.0, '2912': 0.0, '3008': 0.0, '3045': 0.0, '4904': 0.0, '4938': 0.010679218939118496, '5871': 0.03567455262403245, '5880': 0.03281481297736251, '6505': 0.0, '9904': 0.0, '9910': 0.12845676213966795}}
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
validation_years = ["2013to2014", "2014to2015", "2015to2016", "2016to2017", "2017to2018", "2018to2019"]
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2012to2013.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2013to2014.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2014to2015.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2015to2016.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2016to2017.h5')
model6 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/Ensemble/lstm_ensemble_2017to2018.h5')
models = [model1, model2, model3, model4, model5, model6]
SequentialTestROR = {}
SequentialTestTotalROR = {}
#選定欲回測的model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

start = '2019/06/04'
# end = '2020/06/04'
# end = '2020/09/21'
end = '2020/12/01'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

for model, validation_year in zip(models, validation_years):
  print(validation_year)
  asset_at_the_start = 20 * 10**6
  total_asset_present_value = 0
  ROR_Test = {}
  for stock_num in stockNum:
  # for stock_num in ["1101"]:
    #if TestingWeights[stock_num] == 0.0: continue
    if SequentialTestWeight[validation_year][stock_num] == 0.0: continue
    #先算出之後回測用的價位
    Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
    Return_Test_Stock_df = Price_df.copy(deep=True)
    Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    stock_time = str(Price_df.index[0]).split(" ")[0]
    url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
    parameter = {
      "dataset": "TaiwanStockMarginPurchaseShortSale",
      "stock_id": str(stock_num),
      "date": stock_time,
    }
    data = requests.get(url, params=parameter)
    data = data.json()
    data = pd.DataFrame(data['data'])
    data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
    data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
    data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
    data = data.set_index('date')
    # 合併這兩個表格
    new_df = Price_df.join(data)
    new_df = new_df.drop(['stock_id'], axis=1)
    new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
    url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
    parameter = {
      "dataset": "InstitutionalInvestorsBuySell",
      "stock_id": str(stock_num),
      "date": stock_time,
    }
    data = requests.get(url, params=parameter)
    data = data.json()
    data = pd.DataFrame(data['data'])
    df = pd.DataFrame()
    z = 0 
    i = 0
    while i < len(data)  :
      a = data['date'][i]
      count = 0
      sell = 0
      buy = 0 
      if i+6 < len(data)-1:
          for j  in range(i, i+6):
              if data['date'][j]== a :
                count += 1
                sell += data['sell'][j]
                buy += data['buy'][j]
      else:
        for j in range(i, len(data)):
          if data['date'][j] == a :
            count += 1
            sell += data['sell'][j]
            buy += data['buy'][j]
      df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
      i+=count
    new_df = new_df.join(df.set_index('date'))
    new_df = new_df.fillna(axis=0,method='ffill')
    new_df = new_df.drop(['Note'], axis=1)
    new_df.reset_index(drop=True, inplace=True)
    for columnName in new_df.columns:
      if str(columnName) == 'y_label' or str(columnName) == 'index': continue
      new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
      days = 5
      ans = [0]*days
      for i in range(days,len(new_df)):
        Pi = new_df['Close'][i]
        Pj = new_df['Close'][i-days]
        if Pi > Pj : y = 1
        else: y = 0
        ans.append(y)
    new_df['y_label']= ans
    new_df = new_df.reset_index()
    new_df.rename(columns={'index':'Date'}, inplace=True)
    time_list = list(new_df['Date'])
    new_df = new_df.drop(['Date'], axis=1)
    new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
    new_df = new_df.drop(['ShortSaleLimit'], axis=1)
    Price_df = Price_df.reset_index()
    Return_Test_Stock_df.reset_index()
    Return_Test_Stock_df = Return_Test_Stock_df[30:]
    # start evaluaion ###############################################
    time_slide = 30 
    y_preds_list=[0]*(time_slide-1)
    # y_truth_list=[]
    #left_money = asset_at_the_start * TestingWeights[stock_num] #10**6
    left_money = asset_at_the_start * SequentialTestWeight[validation_year][stock_num]
    own_asset = left_money 
    own_stock = 0 
    money_record = [left_money]*time_slide
    origin_hold_stock = 0 
    for i in range(time_slide, len(new_df)): 
      tmp = new_df[i-time_slide:i]
      for col in new_df.columns:
        if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
        mean = statistics.mean(tmp[str(col)])
        stdev = statistics.stdev(tmp[str(col)])
        if stdev == 0: stdev = 1
        def test(s): 
          return (float(s)-mean)/stdev
        tmp[str(col)].map(test)
      test_x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
      predict = int(classes[np.argmax(model.predict(test_x.reshape(1,30,23)))])# + int(classes[np.argmax(model2.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(test_x.reshape(1,30,23)))]) + int(classes[np.argmax(model6.predict(test_x.reshape(1,30,23)))])
      #if predict >= 3 : predict = 1
      #else: predict = 0
      # y_truth_list.append(new_df['y_label'][i])
      y_preds_list.append(predict)
    for i in range(time_slide, len(new_df)): 
      price = int(Price_df['Close'][i]*100)/100
      if y_preds_list[i-4] == 1:
        if own_stock == 0:
          own_stock += int( left_money / ( price * (1 + 0.001425)) )
          left_money = left_money - own_stock*price*(1 + 0.001425)
      elif y_preds_list[i-4] == 0:
        if own_stock > 0:
          left_money += own_stock*price*(1 - 0.001425 - 0.003)
          own_stock = 0
      money_record.append(left_money+own_stock*price)
    total_asset_present_value += int(money_record[-1])
    ROR_Test[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
    print(stock_num)
    print(ROR_Test[stock_num])
  #print("Total ROR：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")
  #print(ROR_Test)
  SequentialTestROR[validation_year] = ROR_Test
  SequentialTestTotalROR[validation_year] = ((total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start )/100
  print(SequentialTestROR[validation_year])
  print(SequentialTestTotalROR[validation_year])
print("========================")
print(SequentialTestROR)
print(SequentialTestTotalROR)

2013to2014
1101
-0.12417136768859129
1301
-0.3091960909024223
1402
-0.0318134769648952
2002
0.03611065906136719
2308
0.703431936952901
2317
0.04695506242064603
2327
0.034101154509539065
2352
0.5218839308633648
2357
0.027181541909875682
2395
0.060009147838562
2454
0.34599466037437465
2474
-0.32719112300741854
2887
-0.11661712994760041
2890
-0.1638187898261805
2891
-0.2445141481797617
3008
-0.20223703262089315
4938
-0.03899522610510392
9904
-0.41696324875083024
9910
0.146659036872664
{'1101': -0.12417136768859129, '1301': -0.3091960909024223, '1402': -0.0318134769648952, '2002': 0.03611065906136719, '2308': 0.703431936952901, '2317': 0.04695506242064603, '2327': 0.034101154509539065, '2352': 0.5218839308633648, '2357': 0.027181541909875682, '2395': 0.060009147838562, '2454': 0.34599466037437465, '2474': -0.32719112300741854, '2887': -0.11661712994760041, '2890': -0.1638187898261805, '2891': -0.2445141481797617, '3008': -0.20223703262089315, '4938': -0.03899522610510392, '9904': -0.416963